# Connect To DB

In [10]:
%load_ext sql
%sql mysql://no_pass@localhost:3306/HogwartzAccounting?local_infile=1

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


# Find Student Debt To University

In [11]:
studentid = 41

In [12]:
%%sql
SELECT

(
    -- amount owed by student
    SELECT SUM(COURSE_COST) FROM COURSE WHERE COURSE_ID IN (
        SELECT COURSE_ID FROM STUDENT_COURSE WHERE STUDENT_ID = {studentid}
    )
)
- 
(
    -- amount payed by student to university
    SELECT SUM(AMOUNT) FROM BANK_TRANSFER WHERE PERSON_ID = {studentid} AND OUTGOING = 0
)
+ 
(
    -- amount payed by university to student
    SELECT SUM(AMOUNT) FROM BANK_TRANSFER WHERE PERSON_ID = {studentid} AND OUTGOING = 1
) AS STUDENT_DEBT



 * mysql://no_pass@localhost:3306/HogwartzAccounting?local_infile=1
1 rows affected.


STUDENT_DEBT
-164.79


# Find University Debt To Professor (Professor Credit)

In [13]:
professor_name = '"Quill Annes"'

In [14]:
%%sql
SELECT
prof.PERSON_ID as pid,
(
    -- amount owed by university
    SELECT SUM(crs.PROFESSOR_PAY * prof_crs.WEEKLY_HOURS) 
    FROM COURSE crs 
    JOIN PROFESSOR_COURSE prof_crs 
    ON crs.course_id = prof_crs.course_id
    WHERE prof_crs.PROFESSOR_ID = pid
)
- 
(
    -- amount payed by university to professor
    SELECT SUM(AMOUNT) FROM BANK_TRANSFER WHERE PERSON_ID = pid AND OUTGOING = 1
)
+ 
(
    -- amount payed by professor to university
    SELECT SUM(AMOUNT) FROM BANK_TRANSFER WHERE PERSON_ID = pid AND OUTGOING = 0
) AS PROFESSOR_CREDIT
FROM PERSON per 
JOIN PROFESSOR prof 
ON per.person_id = prof.person_id 
WHERE CONCAT(per.first_name, ' ', per.last_name) = {professor_name}

 * mysql://no_pass@localhost:3306/HogwartzAccounting?local_infile=1
1 rows affected.


pid,PROFESSOR_CREDIT
396,279.29


# Find Courses Taught By "Old" Professors

In [15]:
old_cutoff = '1991-01-01'

In [16]:
%%sql
SELECT
    c.course_name, 
    CONCAT(per.first_name, ' ', per.last_name) AS professor_name, 
    per.date_of_birth
FROM 
    COURSE c
JOIN 
    PROFESSOR_COURSE pc ON c.course_id = pc.course_id
JOIN 
    PROFESSOR p ON pc.professor_id = p.person_id
JOIN 
    PERSON per ON p.person_id = per.person_id
WHERE 
    per.date_of_birth < "{old_cutoff}";

 * mysql://no_pass@localhost:3306/HogwartzAccounting?local_infile=1
22 rows affected.


course_name,professor_name,date_of_birth
Wizard Analytical Skills,Billie Boote,1990-03-08
Wizard Time Management,Billie Boote,1990-03-08
Magical Botany,Billie Boote,1990-03-08
Wizard Harassment,Pearle Bagby,1990-02-02
Wizard Risk Management,Steffie Reggiani,1990-11-21
Wizard Succession Planning,Steffie Reggiani,1990-11-21
Advanced Charms,Steffie Reggiani,1990-11-21
Wizard Laissez-faire Leadership,Dunstan Dot,1990-09-26
Memory Charms,Dunstan Dot,1990-09-26
Basic Magical Cuisine,Xymenes Sogg,1990-03-25


# Find How Profitable A Specific Course Is

In [17]:
course_name = "Magical Geography"

In [18]:
%%sql
SELECT 
    c.course_name,
    c.course_cost * COUNT(sc.student_id) AS total_course_revenue
FROM 
    COURSE c
JOIN 
    STUDENT_COURSE sc ON c.course_id = sc.course_id
WHERE 
    c.course_name = '{course_name}'
GROUP BY 
    c.course_name, c.course_cost;

 * mysql://no_pass@localhost:3306/HogwartzAccounting?local_infile=1
1 rows affected.


course_name,total_course_revenue
Magical Geography,585.10
